<a href="https://colab.research.google.com/github/Juanvargs/SYS/blob/main/midashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit librosa joblib pyngrok yt-dlp

In [ ]:
%%writefile app.py
import streamlit as st
import librosa
import numpy as np
import joblib
import os
import yt_dlp

st.title("🎶 Detector de Género Musical: Reguetón vs Salsa")

modelo_path = "reggaeton_vs_salsa.pkl"

# Verificación de modelo
if not os.path.exists(modelo_path):
    st.error("❌ No se encontró el archivo del modelo.")
    st.stop()

# Cargar modelo
modelo = joblib.load(modelo_path)
st.success("✅ Modelo cargado correctamente.")

# Entrada del usuario
url = st.text_input("🔗 Ingresa el enlace de la canción (YouTube):")

# Función para descargar MP3 desde YouTube
def download_ytvid_as_mp3(video_url, name):
    output_path = os.path.abspath(f"{name}.mp3")
    options = {
        'format': 'bestaudio/best',
        'keepvideo': False,
        'outtmpl': output_path,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }

    with yt_dlp.YoutubeDL(options) as ydl:
        try:
            ydl.download([video_url])
            return output_path
        except Exception as e:
            st.error(f"❌ Error descargando el audio: {e}")
            return None

# Extraer MFCC
def extraer_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=22050)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0).reshape(1, -1)

# Lógica principal
if url:
    with st.spinner("Descargando audio..."):
        mp3_path = download_ytvid_as_mp3(url, "audio")
        if not mp3_path or not os.path.exists(mp3_path):
            st.stop()

    with st.spinner("Extrayendo características..."):
        try:
            features = extraer_mfcc(mp3_path)
        except Exception as e:
            st.error(f"❌ Error al procesar el audio: {e}")
            st.stop()

    with st.spinner("Clasificando género musical..."):
        try:
            pred = modelo.predict(features)[0]
            clases = modelo.classes_
            st.success(f"🎵 Género detectado: **{clases[pred]}**")
        except Exception as e:
            st.error(f"❌ Error al clasificar: {e}")


In [ ]:
# Paso 2: Autenticación de Ngrok con tu token
from pyngrok import ngrok

ngrok.set_auth_token("2y8ReOP8WVMANp23qnojZmq2yH4_5dZ24aw6BG7qwRpf6TLbx")
print("✅ Authtoken de ngrok configurado.")

# Paso 3: Ejecutar Streamlit en segundo plano
import threading
import os
import time

def run_streamlit():
    os.system("streamlit run app.py")

# Lanzar streamlit
thread = threading.Thread(target=run_streamlit)
thread.daemon = True
thread.start()

# Esperar a que streamlit arranque
time.sleep(5)

# Paso 4: Crear túnel ngrok al puerto 8501
public_url = ngrok.connect(8501)
print(f"🚀 Tu app está disponible en:\n{public_url}")
